# Setting up Google Colab Environment

In [1]:
# These libraries are needed for the pygrib library in Colab.
# Note that is needed if you install pygrib using pip.
# If you use conda, the libraries will be installed automatically.
! apt-get install libeccodes-dev libproj-dev

# Install the python packages
! pip install pyproj
! pip install pygrib

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libproj-dev is already the newest version (7.2.1-1~focal0).
libproj-dev set to manually installed.
The following additional packages will be installed:
  libeccodes-data libeccodes0
The following NEW packages will be installed:
  libeccodes-data libeccodes-dev libeccodes0
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 2,609 kB of archives.
After this operation, 54.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libeccodes-data all 2.16.0-1 [1,394 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libeccodes0 amd64 2.16.0-1 [502 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 libeccodes-dev amd64 2.16.0-1 [712 kB]
Fetched 2,609 kB in 1s (1,762 kB/s)
Selecting previously unselected package libeccodes-data.
(Reading database ... 122352 files and directories currently installed.)
Pre

In [2]:
# Uninstall existing shapely
# We will re-install shapely in the next step by ignoring the binary
# wheels to make it compatible with other modules that depend on
# GEOS, such as Cartopy (used here).
! pip uninstall --yes shapely

Found existing installation: shapely 2.0.1
Uninstalling shapely-2.0.1:
  Successfully uninstalled shapely-2.0.1


In [3]:
# To install cartopy in Colab using pip, we need to install the library
# dependencies first.

! apt-get install -qq libgdal-dev libgeos-dev
! pip install shapely --no-cache-dir shapely
! pip install cartopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 57.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 22.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.9 MB/s eta 0:00:00
  Created wheel for cartopy: filename=Cartopy-0.21.1-cp39-cp39-linux_x86_64.whl size=11113652 sha256=88ec0dd832b0b487a960d70ab4377f74ff2f0e8df53191bd4d1dbb83c026384f
  Stored in directory: /root/.cache/pip/wheels/74/b9/f5/2c94acd7cd21480e6cf63169144d7aac3e8d9cf638225ed578
Successfully built cartopy


In [4]:
! pip install pysteps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.2/502.2 kB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jsmin-3.0.1-py3-none-any.whl
  Created wheel for pysteps: filename=pysteps-1.7.2-cp39-cp39-linux_x86_64.whl size=1297830 sha256=445c9574e6609fa181e0e51d15ea83ffc3ae25b7b2937a02b1cd79504b75b22c
  Stored in directory: /root/.cache/pip/wheels/46/16/14/5561628889f07d62400c52d2c8a8d9a49e25b9ce7e1f4c7151
Successfully built pysteps


In [5]:
! pip install h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
! pip install moviepy
! pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Imports

In [8]:
import numpy as np
import h5py
from datetime import datetime
import cv2
import matplotlib.pyplot as plt
from moviepy.video.io.bindings import mplfig_to_npimage
from moviepy.editor import ImageSequenceClip
from scipy.io import loadmat

from pysteps.motion.lucaskanade import dense_lucaskanade
from pysteps.visualization.motionfields import quiver
from pysteps.extrapolation.semilagrangian import extrapolate
from pysteps.verification.spatialscores import fss

# from pysteps.io.archive import find_by_date
"""just copy-pasted the function from source code"""

Pysteps configuration file found at: /usr/local/lib/python3.9/dist-packages/pysteps/pystepsrc



'just copy-pasted the function from source code'

# Pre-Processing

In [9]:
# -*- coding: utf-8 -*-
"""
pysteps.io.archive
==================

Utilities for finding archived files that match the given criteria.

.. autosummary::
    :toctree: ../generated/

    find_by_date
"""

from datetime import datetime, timedelta
import fnmatch
import os


def find_by_date(
    date,
    root_path,
    path_fmt,
    fn_pattern,
    fn_ext,
    timestep,
    num_prev_files=0,
    num_next_files=0,
    silent=False,
):
    """
    List input files whose timestamp matches the given date.

    Parameters
    ----------
    date: datetime.datetime
        The given date.
    root_path: str
        The root path to search the input files.
    path_fmt: str
        Path format. It may consist of directory names separated by '/',
        date/time specifiers beginning with '%' (e.g. %Y/%m/%d) and wildcards
        (?) that match any single character.
    fn_pattern: str
        The name pattern of the input files without extension. The pattern can
        contain time specifiers (e.g. %H, %M and %S).
    fn_ext: str
        Extension of the input files.
    timestep: float
        Time step between consecutive input files (minutes).
    num_prev_files: int
        Optional, number of previous files to find before the given timestamp.
    num_next_files: int
        Optional, number of future files to find after the given timestamp.
    silent: bool
        Optional, whether to suppress all messages from the method.

    Returns
    -------
    out: tuple
        If num_prev_files=0 and num_next_files=0, return a pair containing the
        found file name and the corresponding timestamp as a datetime.datetime
        object. Otherwise, return a tuple of two lists, the first one for the
        file names and the second one for the corresponding timestemps. The lists
        are sorted in ascending order with respect to timestamp. A None value is
        assigned if a file name corresponding to a given timestamp is not found.

    """
    filenames = []
    timestamps = []

    for i in range(num_prev_files + num_next_files + 1):
        curdate = (
            date
            + timedelta(minutes=num_next_files * timestep)
            - timedelta(minutes=i * timestep)
        )
        fn = _find_matching_filename(
            curdate, root_path, path_fmt, fn_pattern, fn_ext, silent
        )
        filenames.append(fn)

        timestamps.append(curdate)

    if all(filename is None for filename in filenames):
        raise IOError("no input data found in %s" % root_path)

    if (num_prev_files + num_next_files) > 0:
        return filenames[::-1], timestamps[::-1]
    else:
        return filenames, timestamps


def _find_matching_filename(
    date, root_path, path_fmt, fn_pattern, fn_ext, silent=False
):
    path = _generate_path(date, root_path, path_fmt)
    fn = None

    if os.path.exists(path):
      #######----------------------###########
        fn = datetime.strftime(date, fn_pattern).upper() + "." + fn_ext
      # added the .upper() here
      #######----------------------###########

        # test for wildcards
        if "?" in fn:
            filenames = os.listdir(path)
            if len(filenames) > 0:
                for filename in filenames:
                    if fnmatch.fnmatch(filename, fn):
                        fn = filename
                        break

        fn = os.path.join(path, fn)

        if os.path.exists(fn):
            fn = fn
        else:
    # made some changes, was not working as intended
            if not silent:
                print(f"file not found: {fn}")
            fn = None
    elif not silent:
        print("path", path, "not found.")

    return fn


def _generate_path(date, root_path, path_format):
    """Generate file path."""
    if not isinstance(date, datetime):
        raise TypeError("The input 'date' argument must be a datetime object")

    if path_format != "":
        sub_path = date.strftime(path_format)
        return os.path.join(root_path, sub_path)
    else:
        return root_path


In [10]:
# made by Prof. Kaushik Gopalan

def h5ToArray(h5f):
    # Location information
    lat = h5f['Latitude'][()] * 0.01
    lon = h5f['Longitude'][()] * 0.01

    # Retrieving Outgoing Longwave Radiation (OLR)
    olr = np.squeeze(h5f['OLR'][()])

    ###################
    # Filter data within location of interest
    ###################

    lat_filtered = lat[(lat >= 0) & (lat < 40) & (lon >= 60) & (lon < 100)]
    lon_filtered = lon[(lat >= 0) & (lat < 40) & (lon >= 60) & (lon < 100)]
    olr_filtered = olr[(lat >= 0) & (lat < 40) & (lon >= 60) & (lon < 100)]

    ###################
    # Grid data into rectangular grid from 0-40N Latitude, 60-100E Longitude
    # with 0.1 deg resolution
    ###################

    olr_grid = np.zeros((400,400))
    cnt = np.zeros((400,400))

    # Generating index values
    lat_ind = np.int32((lat_filtered-0) * 10)
    lon_ind = np.int32((lon_filtered-60) * 10)
    
    np.add.at(olr_grid, (lat_ind, lon_ind), olr_filtered)
    np.add.at(cnt, (lat_ind, lon_ind), 1)

    olr_grid[cnt > 0] = olr_grid[cnt > 0] / cnt[cnt > 0]  
    olr_grid[cnt == 0] = np.nan

    return olr_grid


In [12]:
def OLRtoImages_None(files):
    processed_files = np.zeros((len(files), 400, 400))
    for i in range(len(files)):
        if files[i] is None:
            continue
        olr_values = h5ToArray(h5py.File(files[i]))
        processed_files[i] = olr_values
    
    return processed_files

In [13]:
date = datetime(2022, 1, 10, 0, 0)
root_path = '/content/drive/MyDrive/Nowcasting/olr_data/'
path_fmt = ''
fn_pattern = '3DIMG_%d%b%Y_%H%M_L2B_OLR_V01R00'
fn_ext = 'h5'
timestep = 30
num_prev_files = 0
num_next_files = 48*7 - 1

# more roubst way to loop through data in file
files, dates = find_by_date(date, root_path, path_fmt, fn_pattern, fn_ext, timestep, num_prev_files, num_next_files)

file not found: /content/drive/MyDrive/Nowcasting/olr_data/3DIMG_15JAN2022_0830_L2B_OLR_V01R00.h5
file not found: /content/drive/MyDrive/Nowcasting/olr_data/3DIMG_15JAN2022_0000_L2B_OLR_V01R00.h5
file not found: /content/drive/MyDrive/Nowcasting/olr_data/3DIMG_14JAN2022_0830_L2B_OLR_V01R00.h5
file not found: /content/drive/MyDrive/Nowcasting/olr_data/3DIMG_13JAN2022_0830_L2B_OLR_V01R00.h5


In [14]:
input_files_not_filtered = OLRtoImages_None(files)

# GIF Generation for 6 frames

In [15]:
input_img = input_files_not_filtered.copy()

In [16]:
flow_params = {
    'dense': True, 
    'fd_method': 'blob', 
    'interp_method': 'rbfinterp2d'
}

extrapolate_params = {
    'timesteps': 6,
    'interp_order': 3,  # cubic, least artifacts (highest computational time) ## 1 is default, which is linear ### 0 for highest computation time
    'return_displacement': False,
}

### Final Test

In [17]:
# finding the first index where a file is not available

for none_idx in range(len(files)):
    if files[none_idx] is None:
        break
none_idx
# print(input_img[none_idx])
# print(input_img[none_idx + 1], input_img[none_idx - 1])

161

In [18]:
all_frames_gen_array = []
all_frames_truth_array = []
for i in range(6, none_idx-6):
    motion_field = dense_lucaskanade(input_img[i-6:i], **flow_params)
    extrapolated_frames = extrapolate(input_img[i-1], motion_field, **extrapolate_params)
    all_frames_gen_array.append(extrapolated_frames)
    all_frames_truth_array.append(input_img[i:i+6])

In [19]:
avg_fss = np.mean([fss(all_frames_gen_array[i][j], all_frames_truth_array[i][j], 0.1, 1) for i in range(len(all_frames_gen_array)) for j in range(6)])
avg_fss

0.9942150981236472

In [27]:
rms_1 = np.mean([np.sqrt(np.nanmean(np.square(all_frames_gen_array[i][0] - all_frames_truth_array[i][0]))) for i in range(len(all_frames_gen_array))])
rms_6 = np.mean([np.sqrt(np.nanmean(np.square(all_frames_gen_array[i][5] - all_frames_truth_array[i][5]))) for i in range(len(all_frames_gen_array))])

(rms_1, rms_6)

(8.933088260084807, 21.909136105674424)